In [1]:
from google.colab import drive 
drive.mount('/content/gdrive')
import numpy as np
import pandas as pd
from scipy import stats
from itertools import product
from matplotlib import pyplot as plt
%matplotlib inline

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
MERGE_TABLE_STORE_4600 = pd.read_csv('/content/gdrive/My Drive/A Reinforcement Learning Approach for Inventory Optimization in Retail/MERGE_TABLE_STORE_4600/MERGE_TABLE_STORE_4600.csv', sep=';', decimal='.')
MERGE_TABLE_STORE_4600['Timestamp'] = pd.to_datetime(MERGE_TABLE_STORE_4600['curr_date'], format='%d%b%Y')
MERGE_TABLE_STORE_4600['curr_date'] = MERGE_TABLE_STORE_4600['Timestamp']
MERGE_TABLE_STORE_4600 = MERGE_TABLE_STORE_4600.set_index(['Timestamp'], drop=True)

shops_4600 = np.sort(MERGE_TABLE_STORE_4600['store_id'].unique())
products_4600 = np.sort(MERGE_TABLE_STORE_4600['product_id'].unique())

sl_data = pd.read_csv('/content/gdrive/My Drive/A Reinforcement Learning Approach for Inventory Optimization in Retail/MERGE_TABLE_STORE_4600/data/echelon/echelon_1_sl.csv', sep=';')
sl_data['date_from'] = pd.to_datetime(sl_data['date_from'], format='%d%b%Y')
sl_data['date_to'] = pd.to_datetime(sl_data['date_to'], format='%d%b%Y')

products_echelon_1 = np.sort(sl_data['product_ids'].unique())
locations_echelon_1 = np.sort(sl_data['location_ids'].unique())

empty_states = pd.read_csv('/content/gdrive/My Drive/A Reinforcement Learning Approach for Inventory Optimization in Retail/MERGE_TABLE_STORE_4600/empty_states.csv', index_col=0)
not_empty_states = pd.read_csv('/content/gdrive/My Drive/A Reinforcement Learning Approach for Inventory Optimization in Retail/MERGE_TABLE_STORE_4600/not_empty_states.csv', index_col=0)
demand_data = pd.read_csv('/content/gdrive/My Drive/A Reinforcement Learning Approach for Inventory Optimization in Retail/MERGE_TABLE_STORE_4600/demand_data.csv', index_col=0)

# Обновление данных среды

Order(SKU, ТТ)_t - это количество товаров,отправка которого инициировано соответствующим Recommended_Order(SKU, ТТ)_t, которое финально доедет и будет выставлено на полку в магазин.

Order(SKU, ТТ)_t и Recommended_Order_t в этих обозначениях _t - дата, когда сформировались данные объёмы.

State: (Stock(SKU, ТТ)_j, Sales(SKU, ТТ)_j, Demand(SKU, ТТ)_j, Orders_Pipeline(SKU, ТТ): [Order_t, Ordrer_t-n, Order_t-m, Order_t-p], n<m<p) period t1-t2


Важно: в State продаж (Sales) и остатки (Stock) - модельные (нужно моделировать. Те, Sales и Stock, что пришли в данных от заказчика (как результат работы реальной системы) - нужны только для восстановления спрос

*   Demand(SKU)_t = F(Sales_all_data, Inv_all_history)
*   Моделирование Sales_t = MAX(Demand_t, Inv_{t-1}+Orders_{t-LT+})
*   Моделирование Inv_t = MAX(Inv_{t-1} + Orders_{t-LT+} - Demand_t, 0)

Рекомендация к заказу: Recommend_Order_t = OUL - (INV + Orders_t - 1), IF Inv_t <= ROL 0, otherwise.

recommend_order - наша рекомендация к заказу
<br>order - объем получаемого заказа

order = F(recommend_order_t, alpha_order)
<br>1)order <= recommend_order_t 
<br>2)$\sum_{sku}$ E(order_{sku,t}) = alpha_order * E(recommend_order_{sku,t})

In [0]:
def sales_update(shop_id, product_id, LT):
    sales = max(demand_data[(demand_data['shop_id']==shop_id) & (demand_data['product_id']==product_id)].iloc[0].demand,
                env.iloc[-1].stock + env.iloc[-LT].order)
    return sales


def stock_update(shop_id, product_id, LT):
    stock = max(env.iloc[-1].stock + env.iloc[-LT].order - demand_data[(demand_data['shop_id']==shop_id) & (demand_data['product_id']==product_id)].iloc[0].demand, 0)
    return stock


def order_update(shop_id, product_id, OUL, ROL):    
    if env.iloc[-1].stock <= ROL:
        rec_order = max(OUL - env.iloc[-1].stock - env.iloc[-1].order, 0)
    else:
        rec_order = 0

    alpha_order = 0.3
    order = np.random.binomial(n=rec_order, p=alpha_order)
    return order

In [31]:
sales_update(4600, 23100, 1)

1.0

# State

В качестве начального состояния можно брать состояние реальной среды

In [0]:
def initial_state(product_id, shop_id):
    state = pd.DataFrame(columns=['sku', 'location', 'sales', 'stock', 'sl', 'order'])

    store_sales_t = MERGE_TABLE_STORE_4600[MERGE_TABLE_STORE_4600['store_id']==shop_id]
    store_sales_t = store_sales_t[store_sales_t['product_id']==product_id].fillna(0)
    store_sales_t = store_sales_t.drop(columns=['curr_date', 'flg_spromo'])

    sl_data_t = sl_data.set_index(['date_from'], drop=True)
    sl_data_t = sl_data_t[sl_data_t['location_ids']==shop_id]
    sl_data_t = sl_data_t[sl_data_t['product_ids']==product_id]

    if not sl_data_t.empty:
        all_days = pd.date_range(start=sl_data_t.index.min(), end=sl_data_t['date_to'].max())
        sl_data_t.index = pd.DatetimeIndex(sl_data_t.index)
        sl_data_t = sl_data_t.reindex(all_days, method='ffill')
        sl_data_t = sl_data_t.drop(columns=['date_to'])

        state_t = pd.merge(store_sales_t, sl_data_t['value'], how='right', left_on=store_sales_t.index, right_on=sl_data_t.index)
        state_t = state_t.rename(columns={'product_id': 'sku', 'store_id': 'location', 's_qty': 'sales', 'value': 'sl'})
        state_t = state_t.drop(columns=['key_0'])
        state_t = state_t.drop_duplicates()
        state_t = state_t.dropna()
        state_t['order'] = 0
        state = state.append(state_t, ignore_index=True)

    return state

In [97]:
%%time
state = initial_state(18973900, 4600)

CPU times: user 589 ms, sys: 10 ms, total: 599 ms
Wall time: 599 ms


In [98]:
state

,sku,location,sales,stock,sl,order
0,18973900,4600,0.0,2.0,0.70,0
1,18973900,4600,0.0,2.0,0.90,0
2,18973900,4600,0.0,2.0,0.65,0


In [0]:
def calculate_states(shops_4600, products_4600, locations_echelon_1, products_echelon_1):
    empty_states = pd.DataFrame(columns=['shop_id', 'product_id'])
    not_empty_states = pd.DataFrame(columns=['shop_id', 'product_id'])

    for shop_id in [shop for shop in shops_4600 if shop in locations_echelon_1]:
        for product_id in [prod for prod in products_4600 if prod in products_echelon_1]:
            state = initial_state(product_id, shop_id)
            if state.empty:
                empty_states = empty_states.append({'shop_id': shop_id, 'product_id': product_id}, ignore_index=True)
            else:
                not_empty_states = not_empty_states.append({'shop_id': shop_id, 'product_id': product_id}, ignore_index=True)

    empty_states_csv = empty_states.to_csv('/content/gdrive/My Drive/A Reinforcement Learning Approach for Inventory Optimization in Retail/MERGE_TABLE_STORE_4600/empty_states.csv')
    not_empty_states_csv = not_empty_states.to_csv('/content/gdrive/My Drive/A Reinforcement Learning Approach for Inventory Optimization in Retail/MERGE_TABLE_STORE_4600/not_empty_states.csv')

calculate_states(shops_4600, products_4600, locations_echelon_1, products_echelon_1)

# Demand Restoring

In [0]:
def demand_restoring(shop_id, product_id):
    # Агрегируем на всякий случай остатки на одну дату (берём максимально известный остаток за дату)
    iv_ts = MERGE_TABLE_STORE_4600.reset_index().groupby('Timestamp').agg({'stock':np.max})

    # Добавим пропущенные даты и заполним значения остатков в эти даты нулями
    iv_ts = iv_ts.reindex(pd.date_range(np.min(iv_ts.index), np.max(iv_ts.index))).fillna(method='ffill')

    # Добавим данные о продажах
    sales_ts = MERGE_TABLE_STORE_4600[(MERGE_TABLE_STORE_4600['product_id'] == product_id) & (MERGE_TABLE_STORE_4600['store_id'] == shop_id)]

    # Объединим продажи и остатки в один датафрейм
    iv_sales = sales_ts[['s_qty']].merge(iv_ts, how='right', left_index=True, right_index=True)
    
    # Удалим начало и "хвост" истории, когда продажи и остатки были равны нулю
    positive_iv_sales = iv_sales[iv_sales.max(axis=1) > 0]
    life_start_date = positive_iv_sales.index[0]
    life_end_date = positive_iv_sales.index[-1]
    iv_sales = iv_sales[(iv_sales.index >= life_start_date) & (iv_sales.index <= life_end_date)] 

    # Дни, когда и остатки и продажи = нулю
    zero_idx = (iv_sales['stock'] == 0) & (iv_sales['s_qty'] == 0)
    # Дни, когда продажи = остаткам
    sales_equal_inv_idx = (iv_sales['stock'] == iv_sales['s_qty']) 
    # Дни, когда продажи >= остаткам
    sales_greater_i_idx = (iv_sales['stock'] <= iv_sales['s_qty']) 

    demand_data = pd.DataFrame(columns=['shop_id', 'product_id', 'lambda', 'demand'])

    iv_sales['weights'] = [1 for x in iv_sales.index]
    
    # Суммарные продажи товара в данном магазине
    iv_sales['scalar'] = iv_sales['s_qty'][(~zero_idx)] * iv_sales['weights'][(~zero_idx)]
    sum_k = iv_sales['scalar'].sum()

    # Количество дней, когда продажи были меньше остатка
    n_k_less_m = iv_sales['weights'][(~zero_idx) & (~sales_greater_i_idx)].sum()

    # Количество дней, когда продажи были равны остатку (все распродано)
    n_k_equal_m = iv_sales['weights'][(~zero_idx) & sales_greater_i_idx].sum()
    
    alpha = 100
    LAMBDA = sum_k / (n_k_less_m + alpha * n_k_equal_m)


    max_sales = MERGE_TABLE_STORE_4600[(MERGE_TABLE_STORE_4600['store_id']==shop_id) &
                                           (MERGE_TABLE_STORE_4600['product_id']==product_id)]['s_qty'].max()
    demand = max(stats.poisson.ppf(0.99, LAMBDA), max_sales, 1)


    demand_data = demand_data.append({'shop_id': shop_id, 'product_id': product_id, 'lambda': LAMBDA, 'demand': demand}, ignore_index=True)

    demand_data[['shop_id', 'product_id', 'demand']] = demand_data[['shop_id', 'product_id', 'demand']].astype('int')

    return demand_data

In [0]:
%%time
demand_restoring(4600, 18973900)

CPU times: user 622 ms, sys: 232 ms, total: 854 ms
Wall time: 888 ms


,shop_id,product_id,lambda,demand
0,4600,18973900,0.002761,1.0


In [0]:
def calculate_demand(not_empty_states):
    demand_data = pd.DataFrame(columns=['shop_id', 'product_id', 'lambda', 'demand'])

    for index, row in not_empty_states.iterrows():
        shop_id = row['shop_id']
        product_id = row['product_id']
        demand_data = demand_data.append(demand_restoring(shop_id, product_id), ignore_index=True)

    demand_data_csv = demand_data.to_csv('/content/gdrive/My Drive/A Reinforcement Learning Approach for Inventory Optimization in Retail/MERGE_TABLE_STORE_4600/demand_data.csv')

calculate_demand(not_empty_states)

# Action

Каким может быть список допустимых Action A_(SKU, Location):
* Set of Order UpTo Level = {0, …, MAX_Demand} 
<br>$\times$
* Set of Re-Order Level = {0, ORDER_UpTo_Level-1}

Как оценить MAX_Demand (t):
* Вариант 1: max(Sales_1, …, Sales_t)
* Вариант 2: max(Poisson (lambda*, quantile=0.99), Вариант 1, 1)

In [0]:
def product_condition(state_range, OUL, ROL):
    prods = []
    for prod in list(product(state_range, OUL, ROL)):
        if prod[1] > prod[2]:
            prods.append(prod)
    return prods

def initial_action(product_id, shop_id, state_range):
    action = pd.DataFrame(columns=['sku', 'location', 'state', 'OUL', 'ROL'])

    max_demand = int(demand_data[(demand_data['product_id'] == product_id) & (demand_data['shop_id'] == shop_id)].iloc[0].demand)

    OUL = [i for i in range(max_demand + 1)]
    ROL = [i for i in range(max(OUL))]
    
    action_t = pd.DataFrame(product_condition(state_range, OUL, ROL), columns=['state', 'OUL', 'ROL'])

    action = pd.merge(action, action_t, how='outer')
    values = {'sku': product_id, 'location': shop_id}
    action = action.fillna(value=values)
    action['R'] = 0.0
    return action

In [99]:
%%time
action = initial_action(18973900, 4600, [i for i in range(len(state))])

CPU times: user 13.9 ms, sys: 0 ns, total: 13.9 ms
Wall time: 13.7 ms


In [100]:
action

,sku,location,state,OUL,ROL,R
0,18973900,4600,0,1,0,0.0
1,18973900,4600,1,1,0,0.0
2,18973900,4600,2,1,0,0.0


# SMART algorithm

<img src="https://github.com/aromanenko/RLIO/blob/master/SMART%20algorithm.png?raw=true" width=80%>



In [107]:
%%time
MAX_STEPS = 100
LT = 1

state_data = pd.DataFrame(columns=['sku', 'location', 'sales', 'stock', 'sl', 'order'])
action_data = pd.DataFrame(columns=['sku', 'location', 'state', 'OUL', 'ROL', 'R'])
env_data = pd.DataFrame(columns=['shop_id', 'product_id', 'order', 'sales', 'stock'])


# for index, row in not_empty_states.iterrows():
    # shop_id = row['shop_id']
    # product_id = row['product_id']
for index, row in not_empty_states.loc[:0].iterrows():
    shop_id = 4600
    product_id = 25100
    # product_id = 18973900

    state = initial_state(product_id, shop_id)
    action = initial_action(product_id, shop_id, [i for i in range(len(state))])
    env = pd.DataFrame(columns=['shop_id', 'product_id', 'order', 'sales', 'stock'])

    for starting_state in range(len(state)):
        m = 0 # time step
        T = 0 # total time
        C = 0 # cumulative reward / total reward
        p = 0 # reward rate / average reward

        while m < MAX_STEPS:
            if m == 0:
                i = starting_state
                alpha = 0.01
                probability = 1

                env = env.append({'shop_id': shop_id,
                                  'product_id': product_id,
                                  'order': state.at[i, 'order'],
                                  'sales': state.at[i, 'sales'],
                                  'stock': state.at[i, 'stock']},
                                 ignore_index=True)
            else:
                alpha = 0.01 / m
                probability = 0.7 / m
            

            non_exploratoty = np.random.binomial(n=1, p=1-probability)
            if non_exploratoty:
                action_index = action[action['state']==i].R.idxmax()
            else:
                action_index = action[action['state']==i].R.idxmax()
                if len(action[action['state']==i]) > 1:
                    action_index = np.random.choice(action[(action['state']==i) & (action.index != action_index)].index, 1)[0]

            
            env = env.append({'shop_id': shop_id,
                              'product_id': product_id,
                              'order': order_update(shop_id, product_id, action.loc[action_index].OUL, action.loc[action_index].ROL),
                              'sales': sales_update(shop_id, product_id, LT),
                              'stock': stock_update(shop_id, product_id, LT)},
                             ignore_index=True)


            new_state = pd.DataFrame(columns=['sku', 'location', 'sales', 'stock', 'sl', 'order'])
            new_state = new_state.append({'sku': product_id,
                                          'location': shop_id,
                                          'sales': env.iloc[-1].sales,
                                          'stock': env.iloc[-1].stock,
                                          'sl': state.at[i, 'sl'],
                                          'order': env.iloc[-1].order,
                                          }, ignore_index=True)


            if state[(state.sales==new_state.sales[0]) & (state.stock==new_state.stock[0]) &
                    (state.sl==new_state.sl[0]) & (state.order==new_state.order[0])].empty:
                state = state.append(new_state, ignore_index=True)
                j = state.index.max()
                action = action.append(initial_action(product_id, shop_id, [j]), ignore_index=True)
            else:
                j = state[(state.sales==new_state.sales[0]) &
                        (state.stock==new_state.stock[0]) &
                        (state.sl==new_state.sl[0]) &
                        (state.order==new_state.order[0])].index[0]


            r = (new_state['sales'] - new_state['stock'] * (1 - new_state['sl']) / new_state['sl']).sum()
            action.at[action_index, 'R'] = (1 - alpha) * action.at[action_index, 'R'] + alpha * max(r - p * LT + action[action['state']==j].R.max(), 0)


            if non_exploratoty:
                C = C + r
                T += LT
                p = C / T
            i = j
            m += 1

    state_data = state_data.append(state)
    action_data = action_data.append(action)
    env_data = env_data.append(env)

    print(shop_id, product_id, C, T, p)

4600 25100 194.0 97 2.0
CPU times: user 11.8 s, sys: 27.2 ms, total: 11.8 s
Wall time: 11.7 s


In [108]:
state_data

,sku,location,sales,stock,sl,order
0,25100,4600,0.0,3.0,0.95,0
1,25100,4600,0.0,3.0,0.90,0
2,25100,4600,0.0,5.0,0.81,0
3,25100,4600,0.0,5.0,0.95,0
4,25100,4600,1.0,5.0,0.95,0
5,25100,4600,0.0,4.0,0.95,0
6,25100,4600,0.0,4.0,0.90,0
7,25100,4600,3.0,1.0,0.95,0
8,25100,4600,2.0,0.0,0.95,0
9,25100,4600,2.0,0.0,0.95,1


In [109]:
action_data

,sku,location,state,OUL,ROL,R
0,25100,4600,0,1,0,0.000000
1,25100,4600,0,2,0,0.000000
2,25100,4600,0,2,1,0.029474
3,25100,4600,1,1,0,0.000000
4,25100,4600,1,2,0,0.028889
...,...,...,...,...,...,...
61,25100,4600,20,2,0,0.000000
62,25100,4600,20,2,1,0.000000
63,25100,4600,21,1,0,0.020100
64,25100,4600,21,2,0,0.000000


In [110]:
env_data

,shop_id,product_id,order,sales,stock
0,4600.0,25100.0,0.0,0.0,3.0
1,4600.0,25100.0,0.0,3.0,1.0
2,4600.0,25100.0,0.0,2.0,0.0
3,4600.0,25100.0,0.0,2.0,0.0
4,4600.0,25100.0,0.0,2.0,0.0
...,...,...,...,...,...
702,4600.0,25100.0,0.0,2.0,0.0
703,4600.0,25100.0,1.0,2.0,0.0
704,4600.0,25100.0,0.0,2.0,0.0
705,4600.0,25100.0,0.0,2.0,0.0
